In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# python에서 파일 관리해주는 라이브러리
import os
import glob
import shutil

In [ ]:
import numpy as np
import pandas as pd

# dacon raw data processing

In [ ]:
!unzip -qq "/content/drive/MyDrive/235743_구내식당 식사 인원 예측 AI 경진대회_data.zip" -d "/content/drive/MyDrive/data"

In [ ]:
train_raw = pd.read_csv('/content/drive/MyDrive/data/train.csv')
test_raw = pd.read_csv('/content/drive/MyDrive/data/test.csv')

In [ ]:
train_raw.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0


In [ ]:
test_raw.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,월,2924,108,154,538,322.0,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,화,2924,62,186,455,314.0,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [ ]:
train_raw = train_raw[['일자', '조식메뉴', '중식메뉴', '석식메뉴']]
test_raw = test_raw[['일자', '조식메뉴', '중식메뉴', '석식메뉴']]

In [ ]:
train_raw.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ..."
1,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무..."
2,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기..."
3,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포..."
4,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:..."


In [ ]:
len(train_raw)

1205

In [ ]:
test_raw.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치
3,2021-02-01,모닝롤/촉촉한치즈케익 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 두부김칫국 새우완...,쌀밥/흑미밥/찰현미밥 배추들깨국 오리대패불고기 시금치프리타타 부추고추장무침 포기김치...,흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치
4,2021-02-02,모닝롤/토마토샌드 우유/주스 계란후라이/찐계란 채소죽/흑미밥 호박맑은국 오이생채 양...,쌀밥/팥밥/찰현미밥 부대찌개 닭살데리야끼조림 버섯탕수 세발나물무침 알타리김치/사과푸...,흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치


In [ ]:
len(test_raw)

50

In [ ]:
raw_data = pd.concat([train_raw, test_raw]).reset_index(drop=True)

In [ ]:
raw_data.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ..."
1,2016-02-02,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무..."
2,2016-02-03,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기..."
3,2016-02-04,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포..."
4,2016-02-05,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:..."


In [ ]:
raw_data.tail()

,일자,조식메뉴,중식메뉴,석식메뉴
1250,2021-04-05,모닝롤/커피콩빵 우유/주스 계란후라이 누룽지탕/흑미밥 청양콩나물국 스팸구이 양상추샐...,쌀밥/흑미밥/찰현미밥 쇠고기미역국 춘천닭갈비 오지치즈후라이 가지두반장볶음 포기김치 ...,흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기
1251,2021-04-06,모닝롤/모닝샌드 우유/주스 계란후라이 고구마스프/흑미밥 아욱국 참치채소볶음 양상추샐...,쌀밥/귀리밥/찰현미밥 순두부백탕 매콤소갈비찜 깻잎완자전 돌나물초장무침 포기김치 시리...,추가밥 짬뽕*생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치
1252,2021-04-07,모닝롤/호떡맥모닝 우유/주스 계란후라이 팥죽/흑미밥 닭살해장국 우엉채조림 양상추샐러...,쌀밥/흑미밥/찰현미밥 냉이국 돈육간장불고기 비빔냉면 오이나물볶음 겉절이김치 양상추샐...,단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치
1253,2021-04-08,모닝롤/크로크무슈 우유/주스 계란후라이 누룽지탕/흑미밥 감자국 두부양념조림 양상추샐...,쌀밥/옥수수밥/찰현미밥 맑은떡국 (New)로제찜닭 가자미구이*장 유채나물무침 포기김...,흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음
1254,2021-04-09,모닝롤/토마토샌드 우유/주스 계란후라이 채소죽/흑미밥 대구지리 애호박나물볶음 양상추...,쌀밥/흑미밥/찰현미밥 사골우거지국 해물누룽지탕 청포묵*양념간장 비름나물고추장무침 석...,흑미밥 맑은버섯국 매운사태조림 춘권*타르타르D 열무나물무침 포기김치


In [ ]:
len(raw_data)

1255

## data check

### 조식메뉴

In [ ]:
for i in range(5):
  print(raw_data['조식메뉴'][i])

모닝롤/찐빵  우유/두유/주스 계란후라이  호두죽/쌀밥 (쌀:국내산) 된장찌개  쥐어채무침  포기김치 (배추,고추가루:국내산) 
모닝롤/단호박샌드  우유/두유/주스 계란후라이  팥죽/쌀밥 (쌀:국내산) 호박젓국찌개  시래기조림  포기김치 (배추,고추가루:국내산) 
모닝롤/베이글  우유/두유/주스 계란후라이  표고버섯죽/쌀밥 (쌀:국내산) 콩나물국  느타리호박볶음  포기김치 (배추,고추가루:국내산) 
모닝롤/토마토샌드  우유/두유/주스 계란후라이  닭죽/쌀밥 (쌀,닭:국내산) 근대국  멸치볶음  포기김치 (배추,고추가루:국내산) 
모닝롤/와플  우유/두유/주스 계란후라이  쇠고기죽/쌀밥 (쌀:국내산) 재첩국  방풍나물  포기김치 (배추,고추가루:국내산) 


1. 모닝롤/ 뒤에 있는 메뉴: 주로 빵
2. /쌀밥 앞에 있는 메뉴 : 주로 죽
3. 국 종류
4. 반찬 종류

In [ ]:
[i for i in raw_data['조식메뉴'][0].split(' ') if i != '' and i[0] != '(']

['모닝롤/찐빵', '우유/두유/주스', '계란후라이', '호두죽/쌀밥', '된장찌개', '쥐어채무침', '포기김치']

In [ ]:
# 빵
[i for i in raw_data['조식메뉴'][0].split(' ') if i != '' and i[0] != '('][0].split('/')[1]

'찐빵'

In [ ]:
# 죽
[i for i in raw_data['조식메뉴'][0].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

'호두죽'

In [ ]:
# 국
[i for i in raw_data['조식메뉴'][0].split(' ') if i != '' and i[0] != '('][4]

'된장찌개'

In [ ]:
# 반찬
[i for i in raw_data['조식메뉴'][0].split(' ') if i != '' and i[0] != '('][5]

'쥐어채무침'

In [ ]:
print([i for i in raw_data['조식메뉴'][5].split(' ') if i != '' and i[0] != '('][0].split('/')[1])
print([i for i in raw_data['조식메뉴'][5].split(' ') if i != '' and i[0] != '('][3].split('/')[0])
print([i for i in raw_data['조식메뉴'][5].split(' ') if i != '' and i[0] != '('][4].split('/')[0])
print([i for i in raw_data['조식메뉴'][5].split(' ') if i != '' and i[0] != '('][5].split('/')[0])

찐빵
견과류죽
감자찌개
명엽채무침


### 중식메뉴

In [ ]:
for i in range(5):
  print(raw_data['중식메뉴'][i])

쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개  쇠불고기 (쇠고기:호주산) 계란찜  청포묵무침  요구르트  포기김치 (배추,고추가루:국내산) 
쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개  가자미튀김  모둠소세지구이  마늘쫑무침  요구르트  배추겉절이 (배추,고추가루:국내산) 
카레덮밥 (쌀,현미흑미:국내산) 팽이장국  치킨핑거 (닭고기:국내산) 쫄면야채무침  견과류조림  요구르트  포기김치 (배추,고추가루:국내산) 
쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국  주꾸미볶음  부추전  시금치나물  요구르트  포기김치 (배추,고추가루:국내산) 
쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국  돈육씨앗강정 (돼지고기:국내산) 우엉잡채  청경채무침  요구르트  포기김치 (배추,고추가루:국내산) 


1. 국
2. 메인 반찬
3. 서브 반찬

In [ ]:
[i for i in raw_data['중식메뉴'][0].split(' ') if i != '' and i[0] != '(']

['쌀밥/잡곡밥', '오징어찌개', '쇠불고기', '계란찜', '청포묵무침', '요구르트', '포기김치']

In [ ]:
# 국
[i for i in raw_data['중식메뉴'][0].split(' ') if i != '' and i[0] != '('][1]

'오징어찌개'

In [ ]:
# 메인 반찬
[i for i in raw_data['중식메뉴'][0].split(' ') if i != '' and i[0] != '('][2]

'쇠불고기'

In [ ]:
# 서브 반찬
[i for i in raw_data['중식메뉴'][0].split(' ') if i != '' and i[0] != '('][3]

'계란찜'

In [ ]:
print([i for i in raw_data['중식메뉴'][5].split(' ') if i != '' and i[0] != '('][1])
print([i for i in raw_data['중식메뉴'][5].split(' ') if i != '' and i[0] != '('][2])
print([i for i in raw_data['중식메뉴'][5].split(' ') if i != '' and i[0] != '('][3])

시래기국
훈제오리구이
도토리묵무침


### 석식메뉴

In [ ]:
for i in range(5):
  print(raw_data['석식메뉴'][i])

쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장  자반고등어구이  두부조림  건파래무침  포기김치 (김치:국내산) 
콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국  유산슬 (쇠고기:호주산) 아삭고추무침  바나나  포기김치 (배추,고추가루:국내산) 
쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개  황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음  포기김치 (배추,고추가루:국내산) 
미니김밥*겨자장 (쌀,현미흑미:국내산) 우동  멕시칸샐러드  군고구마  무피클  포기김치 (배추,고추가루:국내산) 
쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:국내산) 감자소세지볶음  콩나물무침  포기김치 (배추,고추가루:국내산) 


1. 국
2. 메인 반찬
3. 서브 반찬

In [ ]:
[i for i in raw_data['석식메뉴'][0].split(' ') if i != '' and i[0] != '(']

['쌀밥/잡곡밥', '육개장', '자반고등어구이', '두부조림', '건파래무침', '포기김치']

In [ ]:
# 국
[i for i in raw_data['석식메뉴'][0].split(' ') if i != '' and i[0] != '('][1]

'육개장'

In [ ]:
# 메인 반찬
[i for i in raw_data['석식메뉴'][0].split(' ') if i != '' and i[0] != '('][2]

'자반고등어구이'

In [ ]:
# 서브 반찬
[i for i in raw_data['석식메뉴'][0].split(' ') if i != '' and i[0] != '('][3]

'두부조림'

In [ ]:
print([i for i in raw_data['석식메뉴'][5].split(' ') if i != '' and i[0] != '('][1])
print([i for i in raw_data['석식메뉴'][5].split(' ') if i != '' and i[0] != '('][2])
print([i for i in raw_data['석식메뉴'][5].split(' ') if i != '' and i[0] != '('][3])

맑은국
군만두
과일샐러드


- 조식메뉴 (빵, 죽, 국, 반찬)
- 중식메뉴 (국, 메인 반찬, 서브 반찬)
- 석식메뉴 (국, 메인 반찬, 서브 반찬)

=> 4 x 3 x 3 = 36

## to dataframe

In [ ]:
# 0 1 1
dacon_data1 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data1.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data1.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data1.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data1.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data1.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data1.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,오징어찌개,육개장
1,2016-02-02,단호박샌드,김치찌개,어묵국
2,2016-02-03,베이글,팽이장국,청국장찌개
3,2016-02-04,토마토샌드,쇠고기무국,우동
4,2016-02-05,와플,떡국,차돌박이찌개


In [ ]:
# 0 1 2
dacon_data2 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data2.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data2.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data2.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data2.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data2.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data2.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,오징어찌개,자반고등어구이
1,2016-02-02,단호박샌드,김치찌개,유산슬
2,2016-02-03,베이글,팽이장국,황태양념구이
3,2016-02-04,토마토샌드,쇠고기무국,멕시칸샐러드
4,2016-02-05,와플,떡국,닭갈비


In [ ]:
# 0 1 3
dacon_data3 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data3.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data3.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data3.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data3.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data3.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data3.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,오징어찌개,두부조림
1,2016-02-02,단호박샌드,김치찌개,아삭고추무침
2,2016-02-03,베이글,팽이장국,고기전
3,2016-02-04,토마토샌드,쇠고기무국,군고구마
4,2016-02-05,와플,떡국,감자소세지볶음


In [ ]:
# 0 2 1
dacon_data4 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data4.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data4.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data4.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data4.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data4.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data4.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,쇠불고기,육개장
1,2016-02-02,단호박샌드,가자미튀김,어묵국
2,2016-02-03,베이글,치킨핑거,청국장찌개
3,2016-02-04,토마토샌드,주꾸미볶음,우동
4,2016-02-05,와플,돈육씨앗강정,차돌박이찌개


In [ ]:
# 0 2 2
dacon_data5 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data5.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data5.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data5.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data5.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data5.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data5.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,쇠불고기,자반고등어구이
1,2016-02-02,단호박샌드,가자미튀김,유산슬
2,2016-02-03,베이글,치킨핑거,황태양념구이
3,2016-02-04,토마토샌드,주꾸미볶음,멕시칸샐러드
4,2016-02-05,와플,돈육씨앗강정,닭갈비


In [ ]:
# 0 2 3
dacon_data6 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data6.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data6.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data6.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data6.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data6.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data6.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,쇠불고기,두부조림
1,2016-02-02,단호박샌드,가자미튀김,아삭고추무침
2,2016-02-03,베이글,치킨핑거,고기전
3,2016-02-04,토마토샌드,주꾸미볶음,군고구마
4,2016-02-05,와플,돈육씨앗강정,감자소세지볶음


In [ ]:
# 0 3 1
dacon_data7 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data7.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data7.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data7.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data7.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data7.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data7.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,계란찜,육개장
1,2016-02-02,단호박샌드,모둠소세지구이,어묵국
2,2016-02-03,베이글,쫄면야채무침,청국장찌개
3,2016-02-04,토마토샌드,부추전,우동
4,2016-02-05,와플,우엉잡채,차돌박이찌개


In [ ]:
# 0 3 2
dacon_data8 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data8.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data8.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data8.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data8.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data8.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data8.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,계란찜,자반고등어구이
1,2016-02-02,단호박샌드,모둠소세지구이,유산슬
2,2016-02-03,베이글,쫄면야채무침,황태양념구이
3,2016-02-04,토마토샌드,부추전,멕시칸샐러드
4,2016-02-05,와플,우엉잡채,닭갈비


In [ ]:
# 0 3 3
dacon_data9 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data9.loc[j, '일자'] = raw_data['일자'][j]

  try:
    dacon_data9.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0].split('/')[1]
  except:
    dacon_data9.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][0]

  dacon_data9.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data9.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data9.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,계란찜,두부조림
1,2016-02-02,단호박샌드,모둠소세지구이,아삭고추무침
2,2016-02-03,베이글,쫄면야채무침,고기전
3,2016-02-04,토마토샌드,부추전,군고구마
4,2016-02-05,와플,우엉잡채,감자소세지볶음


In [ ]:
# 3 1 1
dacon_data10 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data10.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data10.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data10.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data10.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data10.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,오징어찌개,육개장
1,2016-02-02,팥죽,김치찌개,어묵국
2,2016-02-03,표고버섯죽,팽이장국,청국장찌개
3,2016-02-04,닭죽,쇠고기무국,우동
4,2016-02-05,쇠고기죽,떡국,차돌박이찌개


In [ ]:
# 3 1 2
dacon_data11 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data11.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data11.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data11.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data11.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data11.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,오징어찌개,자반고등어구이
1,2016-02-02,팥죽,김치찌개,유산슬
2,2016-02-03,표고버섯죽,팽이장국,황태양념구이
3,2016-02-04,닭죽,쇠고기무국,멕시칸샐러드
4,2016-02-05,쇠고기죽,떡국,닭갈비


In [ ]:
# 3 1 3
dacon_data12 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data12.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data12.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data12.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data12.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data12.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,오징어찌개,두부조림
1,2016-02-02,팥죽,김치찌개,아삭고추무침
2,2016-02-03,표고버섯죽,팽이장국,고기전
3,2016-02-04,닭죽,쇠고기무국,군고구마
4,2016-02-05,쇠고기죽,떡국,감자소세지볶음


In [ ]:
# 3 2 1
dacon_data13 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data13.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data13.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data13.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data13.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data13.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,쇠불고기,육개장
1,2016-02-02,팥죽,가자미튀김,어묵국
2,2016-02-03,표고버섯죽,치킨핑거,청국장찌개
3,2016-02-04,닭죽,주꾸미볶음,우동
4,2016-02-05,쇠고기죽,돈육씨앗강정,차돌박이찌개


In [ ]:
# 3 2 2
dacon_data14 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data14.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data14.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data14.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data14.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data14.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,쇠불고기,자반고등어구이
1,2016-02-02,팥죽,가자미튀김,유산슬
2,2016-02-03,표고버섯죽,치킨핑거,황태양념구이
3,2016-02-04,닭죽,주꾸미볶음,멕시칸샐러드
4,2016-02-05,쇠고기죽,돈육씨앗강정,닭갈비


In [ ]:
# 3 2 3
dacon_data15 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data15.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data15.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data15.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data15.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data15.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,쇠불고기,두부조림
1,2016-02-02,팥죽,가자미튀김,아삭고추무침
2,2016-02-03,표고버섯죽,치킨핑거,고기전
3,2016-02-04,닭죽,주꾸미볶음,군고구마
4,2016-02-05,쇠고기죽,돈육씨앗강정,감자소세지볶음


In [ ]:
# 3 3 1
dacon_data16 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data16.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data16.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data16.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data16.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data16.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,계란찜,육개장
1,2016-02-02,팥죽,모둠소세지구이,어묵국
2,2016-02-03,표고버섯죽,쫄면야채무침,청국장찌개
3,2016-02-04,닭죽,부추전,우동
4,2016-02-05,쇠고기죽,우엉잡채,차돌박이찌개


In [ ]:
# 3 3 2
dacon_data17 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data17.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data17.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data17.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data17.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data17.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,계란찜,자반고등어구이
1,2016-02-02,팥죽,모둠소세지구이,유산슬
2,2016-02-03,표고버섯죽,쫄면야채무침,황태양념구이
3,2016-02-04,닭죽,부추전,멕시칸샐러드
4,2016-02-05,쇠고기죽,우엉잡채,닭갈비


In [ ]:
# 3 3 3
dacon_data18 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data18.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data18.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][3].split('/')[0]

  dacon_data18.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data18.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data18.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,호두죽,계란찜,두부조림
1,2016-02-02,팥죽,모둠소세지구이,아삭고추무침
2,2016-02-03,표고버섯죽,쫄면야채무침,고기전
3,2016-02-04,닭죽,부추전,군고구마
4,2016-02-05,쇠고기죽,우엉잡채,감자소세지볶음


In [ ]:
# 4 1 1
dacon_data19 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data19.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data19.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data19.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data19.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data19.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,오징어찌개,육개장
1,2016-02-02,호박젓국찌개,김치찌개,어묵국
2,2016-02-03,콩나물국,팽이장국,청국장찌개
3,2016-02-04,근대국,쇠고기무국,우동
4,2016-02-05,재첩국,떡국,차돌박이찌개


In [ ]:
# 4 1 2
dacon_data20 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data20.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data20.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data20.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data20.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data20.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,오징어찌개,자반고등어구이
1,2016-02-02,호박젓국찌개,김치찌개,유산슬
2,2016-02-03,콩나물국,팽이장국,황태양념구이
3,2016-02-04,근대국,쇠고기무국,멕시칸샐러드
4,2016-02-05,재첩국,떡국,닭갈비


In [ ]:
# 4 1 3
dacon_data21 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data21.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data21.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data21.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data21.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data21.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,오징어찌개,두부조림
1,2016-02-02,호박젓국찌개,김치찌개,아삭고추무침
2,2016-02-03,콩나물국,팽이장국,고기전
3,2016-02-04,근대국,쇠고기무국,군고구마
4,2016-02-05,재첩국,떡국,감자소세지볶음


In [ ]:
# 4 2 1
dacon_data22 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data22.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data22.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data22.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data22.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data22.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,쇠불고기,육개장
1,2016-02-02,호박젓국찌개,가자미튀김,어묵국
2,2016-02-03,콩나물국,치킨핑거,청국장찌개
3,2016-02-04,근대국,주꾸미볶음,우동
4,2016-02-05,재첩국,돈육씨앗강정,차돌박이찌개


In [ ]:
# 4 2 2
dacon_data23 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data23.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data23.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data23.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data23.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data23.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,쇠불고기,자반고등어구이
1,2016-02-02,호박젓국찌개,가자미튀김,유산슬
2,2016-02-03,콩나물국,치킨핑거,황태양념구이
3,2016-02-04,근대국,주꾸미볶음,멕시칸샐러드
4,2016-02-05,재첩국,돈육씨앗강정,닭갈비


In [ ]:
# 4 2 3
dacon_data24 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data24.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data24.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data24.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data24.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data24.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,쇠불고기,두부조림
1,2016-02-02,호박젓국찌개,가자미튀김,아삭고추무침
2,2016-02-03,콩나물국,치킨핑거,고기전
3,2016-02-04,근대국,주꾸미볶음,군고구마
4,2016-02-05,재첩국,돈육씨앗강정,감자소세지볶음


In [ ]:
# 4 3 1
dacon_data25 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data25.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data25.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data25.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data25.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data25.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,계란찜,육개장
1,2016-02-02,호박젓국찌개,모둠소세지구이,어묵국
2,2016-02-03,콩나물국,쫄면야채무침,청국장찌개
3,2016-02-04,근대국,부추전,우동
4,2016-02-05,재첩국,우엉잡채,차돌박이찌개


In [ ]:
# 4 3 2
dacon_data26 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data26.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data26.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data26.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data26.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data26.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,계란찜,자반고등어구이
1,2016-02-02,호박젓국찌개,모둠소세지구이,유산슬
2,2016-02-03,콩나물국,쫄면야채무침,황태양념구이
3,2016-02-04,근대국,부추전,멕시칸샐러드
4,2016-02-05,재첩국,우엉잡채,닭갈비


In [ ]:
# 4 3 3
dacon_data27 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data27.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data27.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][4].split('/')[0]

  dacon_data27.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data27.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data27.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,된장찌개,계란찜,두부조림
1,2016-02-02,호박젓국찌개,모둠소세지구이,아삭고추무침
2,2016-02-03,콩나물국,쫄면야채무침,고기전
3,2016-02-04,근대국,부추전,군고구마
4,2016-02-05,재첩국,우엉잡채,감자소세지볶음


In [ ]:
# 5 1 1
dacon_data28 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data28.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data28.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data28.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data28.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data28.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,오징어찌개,육개장
1,2016-02-02,시래기조림,김치찌개,어묵국
2,2016-02-03,느타리호박볶음,팽이장국,청국장찌개
3,2016-02-04,멸치볶음,쇠고기무국,우동
4,2016-02-05,방풍나물,떡국,차돌박이찌개


In [ ]:
# 5 1 2
dacon_data29 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data29.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data29.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data29.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data29.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data29.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,오징어찌개,자반고등어구이
1,2016-02-02,시래기조림,김치찌개,유산슬
2,2016-02-03,느타리호박볶음,팽이장국,황태양념구이
3,2016-02-04,멸치볶음,쇠고기무국,멕시칸샐러드
4,2016-02-05,방풍나물,떡국,닭갈비


In [ ]:
# 5 1 3
dacon_data30 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data30.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data30.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data30.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]

  try:
    dacon_data30.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data30.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,오징어찌개,두부조림
1,2016-02-02,시래기조림,김치찌개,아삭고추무침
2,2016-02-03,느타리호박볶음,팽이장국,고기전
3,2016-02-04,멸치볶음,쇠고기무국,군고구마
4,2016-02-05,방풍나물,떡국,감자소세지볶음


In [ ]:
# 5 2 1
dacon_data31 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data31.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data31.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data31.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data31.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data31.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,쇠불고기,육개장
1,2016-02-02,시래기조림,가자미튀김,어묵국
2,2016-02-03,느타리호박볶음,치킨핑거,청국장찌개
3,2016-02-04,멸치볶음,주꾸미볶음,우동
4,2016-02-05,방풍나물,돈육씨앗강정,차돌박이찌개


In [ ]:
# 5 2 2
dacon_data32 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data32.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data32.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data32.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data32.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data32.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,쇠불고기,자반고등어구이
1,2016-02-02,시래기조림,가자미튀김,유산슬
2,2016-02-03,느타리호박볶음,치킨핑거,황태양념구이
3,2016-02-04,멸치볶음,주꾸미볶음,멕시칸샐러드
4,2016-02-05,방풍나물,돈육씨앗강정,닭갈비


In [ ]:
# 5 2 3
dacon_data33 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data33.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data33.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data33.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]

  try:
    dacon_data33.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data33.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,쇠불고기,두부조림
1,2016-02-02,시래기조림,가자미튀김,아삭고추무침
2,2016-02-03,느타리호박볶음,치킨핑거,고기전
3,2016-02-04,멸치볶음,주꾸미볶음,군고구마
4,2016-02-05,방풍나물,돈육씨앗강정,감자소세지볶음


In [ ]:
# 5 3 1
dacon_data34 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data34.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data34.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data34.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data34.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][1]
  except:
    pass

dacon_data34.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,계란찜,육개장
1,2016-02-02,시래기조림,모둠소세지구이,어묵국
2,2016-02-03,느타리호박볶음,쫄면야채무침,청국장찌개
3,2016-02-04,멸치볶음,부추전,우동
4,2016-02-05,방풍나물,우엉잡채,차돌박이찌개


In [ ]:
# 5 3 2
dacon_data35 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data35.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data35.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data35.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data35.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][2]
  except:
    pass

dacon_data35.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,계란찜,자반고등어구이
1,2016-02-02,시래기조림,모둠소세지구이,유산슬
2,2016-02-03,느타리호박볶음,쫄면야채무침,황태양념구이
3,2016-02-04,멸치볶음,부추전,멕시칸샐러드
4,2016-02-05,방풍나물,우엉잡채,닭갈비


In [ ]:
# 5 3 3
dacon_data36 = pd.DataFrame(columns = ['일자', '조식메뉴', '중식메뉴', '석식메뉴'])

for j in range(len(raw_data)):
  dacon_data36.loc[j, '일자'] = raw_data['일자'][j]

  dacon_data36.loc[j, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][5].split('/')[0]

  dacon_data36.loc[j, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]

  try:
    dacon_data36.loc[j, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][3]
  except:
    pass

dacon_data36.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,쥐어채무침,계란찜,두부조림
1,2016-02-02,시래기조림,모둠소세지구이,아삭고추무침
2,2016-02-03,느타리호박볶음,쫄면야채무침,고기전
3,2016-02-04,멸치볶음,부추전,군고구마
4,2016-02-05,방풍나물,우엉잡채,감자소세지볶음


In [ ]:
dacon_data = pd.concat([dacon_data1, dacon_data2, dacon_data3, dacon_data4, dacon_data5, dacon_data6, dacon_data7, dacon_data8, dacon_data9,
           dacon_data10, dacon_data11, dacon_data12, dacon_data13, dacon_data14, dacon_data15, dacon_data16, dacon_data17, dacon_data18,
           dacon_data19, dacon_data20, dacon_data21, dacon_data22, dacon_data23, dacon_data24, dacon_data25, dacon_data26, dacon_data27,
           dacon_data28, dacon_data29, dacon_data30, dacon_data31, dacon_data32, dacon_data33, dacon_data34, dacon_data35, dacon_data36]).reset_index(drop=True)

In [ ]:
dacon_data.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,오징어찌개,육개장
1,2016-02-02,단호박샌드,김치찌개,어묵국
2,2016-02-03,베이글,팽이장국,청국장찌개
3,2016-02-04,토마토샌드,쇠고기무국,우동
4,2016-02-05,와플,떡국,차돌박이찌개


In [ ]:
len(dacon_data)

45180

In [ ]:
# for a in (0, 3, 4, 5):
#   for b in range(1, 4):
#     for c in range(1, 4):
#       for k in range(36):
#         for j in range(len(raw_data)):
#           dacon_data.loc[j+1255*k, '일자'] = raw_data['일자'][j]
#           if a == 0:
#             try:
#               dacon_data.loc[j+1255*k, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][a].split('/')[1]
#             except:
#               dacon_data.loc[j+1255*k, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][a]
#           else:
#             dacon_data.loc[j+1255*k, '조식메뉴'] = [i for i in raw_data['조식메뉴'][j].split(' ') if i != '' and i[0] != '('][a].split('/')[0]
#           dacon_data.loc[j+1255*k, '중식메뉴'] = [i for i in raw_data['중식메뉴'][j].split(' ') if i != '' and i[0] != '('][b]
#           try:
#             dacon_data.loc[j+1255*k, '석식메뉴'] = [i for i in raw_data['석식메뉴'][j].split(' ') if i != '' and i[0] != '('][c]
#           except:
#             pass

In [ ]:
dacon_data.to_csv('/content/drive/MyDrive/data/dacon_raw_data.csv', encoding='utf-8-sig')

# dacon raw data to DataFrame

In [ ]:
dacon_data.head()

,일자,조식메뉴,중식메뉴,석식메뉴
0,2016-02-01,찐빵,오징어찌개,육개장
1,2016-02-02,단호박샌드,김치찌개,어묵국
2,2016-02-03,베이글,팽이장국,청국장찌개
3,2016-02-04,토마토샌드,쇠고기무국,우동
4,2016-02-05,와플,떡국,차돌박이찌개


## data check

In [ ]:
print(len(dacon_data['조식메뉴']))
print(len(dacon_data['조식메뉴'].unique()))

45180
769


총 조식메뉴는 45180개, 고유한 메뉴 값은 769개

In [ ]:
print(len(dacon_data['중식메뉴']))
print(len(dacon_data['중식메뉴'].unique()))

45180
1140


총 중식메뉴는 45180개, 고유한 메뉴 값은 1140개

In [ ]:
print(len(dacon_data['석식메뉴']))
print(len(dacon_data['석식메뉴'].unique()))

45180
1217


총 석식메뉴는 45180개, 고유한 메뉴 값은 1217개

In [ ]:
dacon_data.iloc[0]

일자      2016-02-01
조식메뉴            찐빵
중식메뉴         오징어찌개
석식메뉴           육개장
Name: 0, dtype: object

In [ ]:
dacon_data.iloc[1255]

일자      2016-02-01
조식메뉴            찐빵
중식메뉴         오징어찌개
석식메뉴       자반고등어구이
Name: 1255, dtype: object

In [ ]:
dacon_data.iloc[2510]

일자      2016-02-01
조식메뉴            찐빵
중식메뉴         오징어찌개
석식메뉴          두부조림
Name: 2510, dtype: object

In [ ]:
len(dacon_data[dacon_data['중식메뉴']=='육개장'])

252

In [ ]:
dacon_data[dacon_data['중식메뉴']=='육개장']

,일자,조식메뉴,중식메뉴,석식메뉴
51,2016-04-19,토마토샌드,육개장,유부장국
98,2016-06-28,햄야채샌드,육개장,계란국
139,2016-08-25,인절미샌드,육개장,작은밥
178,2016-10-25,햄치즈샌드,육개장,가쯔오장국
230,2017-01-05,토마토샌드,육개장,팽이장국
...,...,...,...,...
37356,2020-01-09,두부양념조림,육개장,양배추샐러드/모닝빵
37430,2020-04-27,사과베이비샐러드,육개장,꿔바로우탕수육*소스
37455,2020-06-04,오이사과무침,육개장,브로컬리오징어숙회
37568,2020-12-04,야채소시지전,육개장,노가리고추조림


## to final dataframe

In [ ]:
menu_arr = np.array(dacon_data[['조식메뉴', '중식메뉴', '석식메뉴']]).flatten()
menu_arr

array(['찐빵', '오징어찌개', '육개장', ..., '애호박나물볶음', '청포묵*양념간장', '춘권*타르타르D'],
      dtype=object)

In [ ]:
len(menu_arr)

135540

### menu tagging

In [ ]:
menu_df = pd.DataFrame({'Menu' : menu_arr})
menu_unique_list = menu_df['Menu'].unique()
print(menu_unique_list)
print(len(menu_unique_list))

['찐빵' '오징어찌개' '육개장' ... '달래오이무침' '비엔나구이*케찹' '애호박나물볶음']
2342


In [ ]:
# {메뉴명 : 메뉴ID}
menu_id_dict = {m : i for i, m in enumerate(menu_unique_list)}
menu_id_dict

{'찐빵': 0,
 '오징어찌개': 1,
 '육개장': 2,
 '단호박샌드': 3,
 '김치찌개': 4,
 '어묵국': 5,
 '베이글': 6,
 '팽이장국': 7,
 '청국장찌개': 8,
 '토마토샌드': 9,
 '쇠고기무국': 10,
 '우동': 11,
 '와플': 12,
 '떡국': 13,
 '차돌박이찌개': 14,
 '시래기국': 15,
 '맑은국': 16,
 '야채샌드': 17,
 '꽃게탕': 18,
 '김치콩나물국': 19,
 '치즈프레즐': 20,
 '시금치국': 21,
 '홍합미역국': 22,
 '마늘빵': 23,
 '된장찌개': 24,
 '참치샌드': 25,
 '냉이된장국': 26,
 '인절미토스트': 27,
 '유부장국': 28,
 '순두부찌개': 29,
 '씨크립샌드': 30,
 '대구찌개': 31,
 '설렁탕': 32,
 '팬케익': 33,
 '쇠고기느타리국': 34,
 '옹심이만두국': 35,
 '콩나물국': 36,
 '콩비지찌개': 37,
 '꿀호떡': 38,
 '열무된장국': 39,
 '크로와상': 40,
 '북어계란국': 41,
 '잔치국수': 42,
 '에그샌드': 43,
 '수제비국': 44,
 '닭개장': 45,
 '프렌치토스트': 46,
 '얼갈이된장국': 47,
 '버섯들깨탕': 48,
 '단팥빵': 49,
 '쇠고기미역국': 50,
 '바나나샌드': 51,
 '근대된장국': 52,
 '사과파이': 53,
 '부대찌개': 54,
 '오징어국': 55,
 '브라우니': 56,
 '고구마샌드': 57,
 '가쯔오장국': 58,
 '롤케익': 59,
 '북어국': 60,
 '햄치즈샌드': 61,
 '고추장찌개': 62,
 '짬뽕': 63,
 '배추된장국': 64,
 '연두부탕': 65,
 '롤케이크': 66,
 '달래된장찌개': 67,
 '쑥국': 68,
 '계란파국': 69,
 '콩나물김치국': 70,
 '뼈해장국': 71,
 '베이컨샌드': 72,
 '사골우거지국': 73,
 '양송이스프': 74,
 '닭곰탕': 75,
 '

### sessionID 생성

In [ ]:
len(menu_arr)

135540

In [ ]:
session_id_list = []

for i in range(4518):
  for j in range(30):
    session_id_list.append(i)

In [ ]:
print(session_id_list)
print(len(session_id_list))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,

### to DataFrame

In [ ]:
dacon_menu_df = pd.DataFrame({'SessionID' : session_id_list,
                              'Menu' : menu_arr})

In [ ]:
dacon_menu_df

,SessionID,Menu
0,0,찐빵
1,0,오징어찌개
2,0,육개장
3,0,단호박샌드
4,0,김치찌개
...,...,...
135535,4517,유채나물무침
135536,4517,채소계란찜
135537,4517,애호박나물볶음
135538,4517,청포묵*양념간장


In [ ]:
dacon_menu_df['MenuID'] = list(dacon_menu_df['Menu'].apply(lambda x:menu_id_dict[x]))

In [ ]:
dacon_menu_df

,SessionID,Menu,MenuID
0,0,찐빵,0
1,0,오징어찌개,1
2,0,육개장,2
3,0,단호박샌드,3
4,0,김치찌개,4
...,...,...,...
135535,4517,유채나물무침,1990
135536,4517,채소계란찜,1517
135537,4517,애호박나물볶음,2341
135538,4517,청포묵*양념간장,1991


In [ ]:
dacon_menu_df.to_csv('/content/drive/MyDrive/data/dacon_menu_final.csv', encoding='utf-8-sig')